# Lab 7 :  Graphical and hidden Markov models
```
- [S23] Advanced Machine Learning, Innopolis University
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
Lab Plan
1. Project start
2. Recap on HMMs
3. Simple example for HMMs
3. Task
```

<hr>


## 1. Recap on HMMs and Likelihood

<!--![](https://www.dropbox.com/scl/fi/wm8nbmqgchv3orw7r488f/Lecture-5-HMM.jpg?rlkey=3x0lmmzxchgz6esmq6yrjfzne&dl=1)-->

In [21]:
import numpy as np

states = ('Sunny', 'Rainy')
observations = ('happy', 'grumpy')
pi = np.array([2. / 3., 1. / 3.])  # initial probability
A = np.array([[7. / 9., 2. / 9.], [0.4, 0.6]])  # Transmission probability
B = np.array([[0.8, 0.2], [0.4, 0.6]])  # Emission probability
bob_says = np.array([0, 0, 1, 1, 1, 0])


def forward(obs_seq, pi, A, B):
    T = len(obs_seq)
    N = A.shape[0]
    alpha = np.zeros((T, N))
    alpha[0] = pi * B[:, obs_seq[0]]
    for t in range(1, T):
        alpha[t] = np.inner(alpha[t - 1], A) * B[:, obs_seq[t]]
    return alpha


def likelihood(alpha):
    # using the forward part of the forward-backward algorithm
    return alpha[-1].sum()


alpha = forward(bob_says, pi, A, B)
print(alpha)

print(likelihood(alpha))

[[0.53333333 0.13333333]
 [0.35555556 0.11733333]
 [0.06052346 0.12757333]
 [0.01508469 0.06045203]
 [0.00503326 0.02538306]
 [0.00764435 0.00689726]]
0.014541607035957256


## 2. Viterbi algorithm simple example


In [22]:
from numpy import random

# Transition Probabilities (s - sunny; r-raining)
p_ss = 7. / 9.
p_sr = 2. / 9.
p_rs = 0.4
p_rr = 0.6

# Initial Probabilities
p_s = 2 / 3
p_r = 1 / 3

# Emission Probabilities (h - happy; g- grumpy)
p_sh = 0.8
p_sg = 0.2
p_rh = 0.4
p_rg = 0.6

moods = ['H', 'H', 'G', 'G', 'G', 'H']
probabilities = []
weather = []

if moods[0] == 'H':
    probabilities.append((p_s * p_sh, p_r * p_rh))
else:
    probabilities.append((p_s * p_sg, p_r * p_rg))

for i in range(1, len(moods)):
    yesterday_sunny, yesterday_rainy = probabilities[-1]
    if moods[i] == 'H':
        today_sunny = max(yesterday_sunny * p_ss * p_sh, yesterday_rainy * p_rs * p_sh)
        today_rainy = max(yesterday_sunny * p_sr * p_rh, yesterday_rainy * p_rr * p_rh)
        probabilities.append((today_sunny, today_rainy))
    else:
        today_sunny = max(yesterday_sunny * p_ss * p_sg, yesterday_rainy * p_rs * p_sg)
        today_rainy = max(yesterday_sunny * p_sr * p_rg, yesterday_rainy * p_rr * p_rg)
        probabilities.append((today_sunny, today_rainy))

for p in probabilities:
    if p[0] > p[1]:
        weather.append('S')
    else:
        weather.append('R')

weather

['S', 'S', 'S', 'R', 'R', 'S']

## 3. Parts of Speech (POS) Tagging

---

* process of assigning a part-of-speech to each word in a text
* POS is a disambiguation task (some words can have multiple part of speech depending in the context)
* Gives an idea about syntactic structure (parsing)

**How does it work?**

<!--![](http://www.cs.virginia.edu/~hw5x/Course/TextMining-2019Spring/_site/docs/codes/HMM.PNG)-->

## 4. Task (Viterbi algorithm)

<!-- ![](https://miro.medium.com/max/1080/1*8-5KZVj-_jZOWN83gGhD5A.png) -->
<!--![](https://www.dropbox.com/scl/fi/twg8yqdhqwxcbr5lcqj4w/pos_tag1.png?rlkey=5kizqwfv8gk8vg26434sp0rk2&dl=1)-->

First steps :
*  Calculate or estimate transition probabilities between different parts of speech tags
* Calculate or estimate prior probabilities of tags


In the task you are going to use a a fraction of conll2000 dataset. The dataset should be retrieved from : [Train](https://www.dropbox.com/s/x9n6f9o9jl7pno8/train_pos.txt?dl=1), [Test](https://www.dropbox.com/s/v8nccvq7jewcl8s/test_pos.txt?dl=1)

1. Calculate the transition probability and emission matrices (First step towards viterbi) <!-- 10 points -->
1. Use Viterbi algorithm for POS tagging task (you can use libiries such as [`hmmlearn`](https://github.com/hmmlearn/hmmlearn)).
1. Test your viterbi model on the test set and record the accuracy. The accuray referes to the number of correcly predicted tags in the whole test samples.
1. Using Recurrent neural network (RNN, GRU or LSTM) or Transfomer solve the task for POS

## First steps:

### Calculate or estimate transition probabilities between different parts of speech tags

In [23]:
# Load dataset train and test
from datasets import load_dataset

dataset = load_dataset('conll2000')

train_data = dataset['train']
test_data = dataset['test']

print(len(train_data))

8937


In [24]:
print(train_data[0]['pos_tags'])

[19, 14, 11, 19, 39, 27, 37, 32, 34, 11, 15, 19, 14, 19, 22, 14, 20, 5, 15, 14, 19, 19, 5, 34, 32, 34, 11, 15, 19, 14, 20, 9, 20, 24, 15, 22, 6]


    - name: pos_tags
      sequence:
        class_label:
          names:
            '0': ''''''
            '1': '#'
            '2': $
            '3': (
            '4': )
            '5': ','
            '6': .
            '7': ':'
            '8': '``'
            '9': CC
            '10': CD
            '11': DT
            '12': EX
            '13': FW
            '14': IN
            '15': JJ
            '16': JJR
            '17': JJS
            '18': MD
            '19': NN
            '20': NNP
            '21': NNPS
            '22': NNS
            '23': PDT
            '24': POS
            '25': PRP
            '26': PRP$
            '27': RB
            '28': RBR
            '29': RBS
            '30': RP
            '31': SYM
            '32': TO
            '33': UH
            '34': VB
            '35': VBD
            '36': VBG
            '37': VBN
            '38': VBP
            '39': VBZ
            '40': WDT
            '41': WP
            '42': WP$
            '43': WRB
    - name: chunk_tags
      sequence:
        class_label:
          names:
            '0': O
            '1': B-ADJP
            '2': I-ADJP
            '3': B-ADVP
            '4': I-ADVP
            '5': B-CONJP
            '6': I-CONJP
            '7': B-INTJ
            '8': I-INTJ
            '9': B-LST
            '10': I-LST
            '11': B-NP
            '12': I-NP
            '13': B-PP
            '14': I-PP
            '15': B-PRT
            '16': I-PRT
            '17': B-SBAR
            '18': I-SBAR
            '19': B-UCP
            '20': I-UCP
            '21': B-VP
            '22': I-VP

In [25]:
pos_tags_dict = {
    '': 0,
    '#': 1,
    '$': 2,
    '(': 3,
    ')': 4,
    ',': 5,
    '.': 6,
    ':': 7,
    '``': 8,
    'CC': 9,
    'CD': 10,
    'DT': 11,
    'EX': 12,
    'FW': 13,
    'IN': 14,
    'JJ': 15,
    'JJR': 16,
    'JJS': 17,
    'MD': 18,
    'NN': 19,
    'NNP': 20,
    'NNPS': 21,
    'NNS': 22,
    'PDT': 23,
    'POS': 24,
    'PRP': 25,
    'PRP$': 26,
    'RB': 27,
    'RBR': 28,
    'RBS': 29,
    'RP': 30,
    'SYM': 31,
    'TO': 32,
    'UH': 33,
    'VB': 34,
    'VBD': 35,
    'VBG': 36,
    'VBN': 37,
    'VBP': 38,
    'VBZ': 39,
    'WDT': 40,
    'WP': 41,
    'WP$': 42,
    'WRB': 43
}


In [26]:
print(train_data.shape[0])

8937


In [27]:
from collections import defaultdict


# Calculate transition probabilities between different part of speach tags
def calculate_transition_probability(data):
    # Dictionary for count tags
    tag_counts = defaultdict(int)
    # Dictionary for count transition between every POS
    transition_counts = defaultdict(lambda: defaultdict(int))

    for features in data:
        # Length sentence
        size = len(features['pos_tags'])

        for index, pos_tag in enumerate(features['pos_tags']):
            if index < size - 1:
                next_pos_tag = features['pos_tags'][index + 1]

                # Calculate count tags in sentences
                tag_counts[pos_tag] += 1

                # Calculate count transition between POS_t and POS_{t+1}
                transition_counts[pos_tag][next_pos_tag] += 1

    # Dictionary for probability transition
    transition_probability = defaultdict(lambda: defaultdict(int))

    # Calculate transition probabilities for every transition pos
    for tag in transition_counts:
        s = 0
        for next_tag in transition_counts[tag]:
            s += transition_counts[tag][next_tag]
            transition_probability[tag][next_tag] = transition_counts[tag][next_tag] / tag_counts[tag]
    return transition_probability

In [28]:
calculate_transition_probability(train_data)

defaultdict(<function __main__.calculate_transition_probability.<locals>.<lambda>()>,
            {19: defaultdict(int,
                         {14: 0.24783459994026483,
                          39: 0.03879467693226695,
                          22: 0.08498987820661733,
                          19: 0.11767829290147015,
                          5: 0.11253443069060498,
                          32: 0.04148275976504165,
                          15: 0.009955862343609996,
                          6: 0.10573125808913815,
                          24: 0.022035641987190123,
                          36: 0.00789831745926393,
                          38: 0.003650482859323665,
                          35: 0.04865098065244085,
                          9: 0.03972389075100388,
                          37: 0.012079779643580127,
                          18: 0.01818604188099426,
                          0: 0.005210234626489231,
                          27: 0.017057710815385124,
           

In [29]:
# Calculate prior probabilities
def calculate_prior_probabilities(data):
    # Dictionary count every POS start in sentence
    start_tag_counts = defaultdict(int)

    # Count total sentences
    total_sentences = 0

    for features in data:
        if features['pos_tags']:
            start_tag = features['pos_tags'][0]
            start_tag_counts[start_tag] += 1
            total_sentences += 1

    # Dictionary prior probabilities for every POS
    prior_probability = defaultdict(int)

    for tag in start_tag_counts:
        prior_probability[tag] = start_tag_counts[tag] / total_sentences

    return prior_probability

In [30]:
calculate_prior_probabilities(train_data)

defaultdict(int,
            {19: 0.045210384959713516,
             20: 0.19192032229185318,
             9: 0.056177260519247985,
             11: 0.21239928379588183,
             8: 0.07273948075201432,
             22: 0.04364368845120859,
             14: 0.12891674127126232,
             25: 0.0583034914950761,
             27: 0.06031781557743957,
             37: 0.005259623992837959,
             15: 0.044874664279319604,
             26: 0.00861683079677708,
             43: 0.006378692927484333,
             34: 0.004811996418979409,
             41: 0.0032452999104744854,
             10: 0.016226499552372427,
             17: 0.004923903312444047,
             32: 0.0033572068039391225,
             36: 0.01342882721575649,
             3: 0.003133393017009848,
             16: 0.0011190689346463742,
             0: 0.0004476275738585497,
             28: 0.002126230975828111,
             35: 0.0011190689346463742,
             12: 0.0032452999104744854,
             39:

## Calculate the transition probability and emission matrices (First step towards viterbi)

In [31]:
# Function translating from dictionary to matrix 
def translate_dict_to_matrix(dictionary, label=None, length_row=None, length_column=None):
    l_row = len(dictionary.keys()) + 1 if length_row is None else length_row + 1
    l_column = len(dictionary.keys()) + 1 if length_column is None else length_column + 1
    print(l_row, l_column)

    matrix = [[0 for _ in range(l_column)] for _ in range(l_row)]

    for tag, next_tags in dictionary.items():
        for next_tag, prob in next_tags.items():
            tag1 = label[tag] if label is not None else tag
            matrix[tag1][next_tag] = prob

    return matrix

In [32]:
transition_probability_dict = calculate_transition_probability(train_data)
transition_probability_matrix = translate_dict_to_matrix(transition_probability_dict)

45 45


In [33]:
transition_probability_matrix

[[0.010731707317073172,
  0,
  0.000975609756097561,
  0.011707317073170732,
  0.000975609756097561,
  0.001951219512195122,
  0.000975609756097561,
  0.011707317073170732,
  0.010731707317073172,
  0.054634146341463415,
  0.002926829268292683,
  0.054634146341463415,
  0,
  0,
  0.13463414634146342,
  0.00975609756097561,
  0,
  0,
  0.011707317073170732,
  0.04975609756097561,
  0.09170731707317073,
  0,
  0.03121951219512195,
  0,
  0,
  0.10439024390243902,
  0.001951219512195122,
  0.00975609756097561,
  0,
  0,
  0,
  0,
  0.021463414634146343,
  0,
  0.002926829268292683,
  0.12,
  0.014634146341463415,
  0.006829268292682927,
  0.004878048780487805,
  0.20195121951219513,
  0.010731707317073172,
  0.002926829268292683,
  0,
  0.006829268292682927,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1.0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],


In [34]:
def calculate_emission_probabilities(data):
    emission_counts = defaultdict(lambda: defaultdict(int))
    for features in data:
        for index, token in enumerate(features['tokens']):
            i = features['pos_tags'][index]
            emission_counts[token][i] += 1
    
    
    emission_probabilities = defaultdict(lambda: defaultdict(int))

    for token, value in emission_counts.items():
        for tag, count in value.items():
            count_word = sum(emission_counts[token].values())
            emission_probabilities[token][tag] = emission_counts[token][tag] / count_word

    return emission_probabilities


In [35]:
def create_dictionary():
    dictionary = set()

    for sentence in train_data['tokens'] + test_data['tokens']:
        for word in sentence:
            dictionary.add(word)
    dictionary = dict.fromkeys(dictionary, 0)
    sorted_dictionary = dict(sorted(dictionary.items()))

    index = 0
    for key in sorted_dictionary:
        sorted_dictionary[key] = index
        index += 1

    return sorted_dictionary

In [36]:
token_dict = create_dictionary()

In [37]:
token_dict

{'!': 0,
 '#': 1,
 '$': 2,
 '%': 3,
 '&': 4,
 "'": 5,
 "''": 6,
 "'40s": 7,
 "'60s": 8,
 "'70s": 9,
 "'71": 10,
 "'80s": 11,
 "'86": 12,
 "'90s": 13,
 "'S": 14,
 "'T-": 15,
 "'d": 16,
 "'ll": 17,
 "'m": 18,
 "'n'": 19,
 "'re": 20,
 "'s": 21,
 "'til": 22,
 "'ve": 23,
 ',': 24,
 '-': 25,
 '--': 26,
 '-LCB-': 27,
 '-LRB-': 28,
 '-RCB-': 29,
 '-RRB-': 30,
 '.': 31,
 '...': 32,
 '.50': 33,
 '0.0015': 34,
 '0.0018': 35,
 '0.01': 36,
 '0.0108': 37,
 '0.03': 38,
 '0.05': 39,
 '0.06': 40,
 '0.1': 41,
 '0.10': 42,
 '0.14': 43,
 '0.15': 44,
 '0.19': 45,
 '0.2': 46,
 '0.23': 47,
 '0.24': 48,
 '0.25': 49,
 '0.272': 50,
 '0.3': 51,
 '0.32': 52,
 '0.36': 53,
 '0.375': 54,
 '0.4': 55,
 '0.43': 56,
 '0.5': 57,
 '0.51': 58,
 '0.53': 59,
 '0.60': 60,
 '0.7': 61,
 '0.71': 62,
 '0.73': 63,
 '0.75': 64,
 '0.76': 65,
 '0.8': 66,
 '0.86': 67,
 '0.88': 68,
 '0.89': 69,
 '0.9': 70,
 '0.92': 71,
 '0.95': 72,
 '1': 73,
 '1,000': 74,
 '1,022,000': 75,
 '1,040': 76,
 '1,050': 77,
 '1,103.11': 78,
 '1,150,000': 79,


In [38]:
emission_probabilities_dict = calculate_emission_probabilities(train_data)
emission_probabilities_matrix = translate_dict_to_matrix(emission_probabilities_dict, token_dict,
                                                         max(token_dict.values()), len(pos_tags_dict))

21589 45


In [39]:
emission_probabilities_dict

defaultdict(<function __main__.calculate_emission_probabilities.<locals>.<lambda>()>,
            {'Confidence': defaultdict(int, {19: 1.0}),
             'in': defaultdict(int,
                         {14: 0.9935256032960565,
                          19: 0.0044143613890523835,
                          27: 0.0020600353148911123}),
             'the': defaultdict(int,
                         {11: 0.9981559822106519, 14: 0.0018440177893480855}),
             'pound': defaultdict(int, {19: 1.0}),
             'is': defaultdict(int,
                         {39: 0.9978798586572438, 22: 0.0021201413427561835}),
             'widely': defaultdict(int, {27: 1.0}),
             'expected': defaultdict(int,
                         {37: 0.9298245614035088, 35: 0.07017543859649122}),
             'to': defaultdict(int, {32: 1.0}),
             'take': defaultdict(int,
                         {34: 0.8712871287128713,
                          38: 0.1188118811881188,
                         

In [40]:
emission_probabilities_matrix

[[0,
  0,
  0,
  0,
  0,
  0,
  1.0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  1.0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1.0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.9991708126036484,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.0008291873963515755,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1.0,
 